# Final project restart: Vivek
Import the modules, assign variables and define functions.

This is the version that I am using since I got some advice from Vivek on the forum. He is really great and added a great customized function for getting the scoring function to work better.

In [174]:
import sys
import pickle
sys.path.append("../tools/")

import pandas as pd
import numpy as np

from feature_format import featureFormat, targetFeatureSplit
from tester import dump_classifier_and_data

import matplotlib.pyplot as plt
%matplotlib inline
from pprint import pprint

from sklearn.preprocessing import Imputer
                                                
from sklearn.pipeline import Pipeline
from sklearn.grid_search import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler

from sklearn.decomposition import PCA

from sklearn.cross_validation import StratifiedShuffleSplit
from sklearn import cross_validation

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB

from tester import test_classifier

from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import precision_recall_fscore_support
pd.set_option('display.max_rows', 200)

### Data Import
Defining functions that I will use for data import and prep.

In [175]:
data = "final_project_dataset.pkl"

In [214]:
def import_data(data):
    '''This are the things I will do to import the data everytime, 
    regardless of what variables I make.'''
    with open(data, "r") as data_file:
        data_dict = pickle.load(data_file)
    df = pd.DataFrame(data_dict)
    df = df.transpose()
    df = df.drop('email_address', axis=1)
    df = df.astype(float)
    df = df.drop('TOTAL')
    df = df.drop("THE TRAVEL AGENCY IN THE PARK")
    df = df.drop("LOCKHART EUGENE E")
    df = df.drop("loan_advances", axis=1)
    return df

In [177]:
def import_data1(data):
    '''This are the things I will do to import the data everytime, 
    regardless of what variables I make.'''
    with open(data, "r") as data_file:
        data_dict1 = pickle.load(data_file)
    return data_dict1

In [178]:
# df1 = import_data1(data)

In [215]:
# df = import_data(data)

In [273]:
def get_features_labels_new(df):
    '''This is where the features and labels are extracted to use as arguments
    for sklearn\'s cross_validation function. It is also where I will do add
    any new variables.'''
    
    #add columns
#     df['deferred_ratio'] = df['deferred_income']/(df['total_payments'] + 1)
#     df['pct_from_poi'] = df['from_poi_to_this_person']/(df['from_messages'] + 1)
#     df['pct_to_poi'] = df['from_this_person_to_poi']/(df['from_messages'] + 1)
#     df['to_from'] = df['pct_from_poi']*df['pct_from_poi']
    
    #drop columns
    df = df.drop('restricted_stock_deferred', axis=1)
    df = df.drop('director_fees', axis=1)
    df = df.drop('restricted_stock', axis=1)

    
    #drop rows
    for i in df.index:
        if df.ix[i].count() < 3:
            df = df.drop(i, axis=0)
    
    features_list = list(df.columns)
    features_list.remove('poi')
    features = df[features_list]
    labels = df['poi']
    return features, labels

In [181]:
def get_features_labels_new1(df):
    '''This is where the features are pared and features list is created for 
    the grader. It is also where I will do add any new variables.'''
    
    #add columns
#     df['deferred_ratio'] = df['deferred_income']/(df['total_payments'] + 1)
#     df['pct_from_poi'] = df['from_poi_to_this_person']/(df['from_messages'] + 1)
#     df['pct_to_poi'] = df['from_this_person_to_poi']/(df['from_messages'] + 1)
#     df['to_from'] = df['pct_from_poi']*df['pct_from_poi']
    
    #drop columns
    df = df.drop('restricted_stock_deferred', axis=1)
    df = df.drop('director_fees', axis=1)
    df = df.drop('restricted_stock', axis=1)
    
    #drop rows
    for i in df.index:
        if df.ix[i].count() < 3:
            df = df.drop(i, axis=0)
    
    features_list = list(df.columns)
    features_list.remove('poi')
    features_list.insert(0,'poi')
#     features = df[features_list]
#     labels = df['poi']
    return df, features_list

In [182]:
def get_features_labels_remove_non_salary(df):
    '''This is where the features and labels are extracted to use as arguments
    for sklearn\'s cross_validation function. It is also where I will do add
    any new variables.'''
    
    #add columns
#     df['deferred_ratio'] = df['deferred_income']/(df['total_payments'] + 1)
#     df['pct_from_poi'] = df['from_poi_to_this_person']/(df['from_messages'] + 1)
#     df['pct_to_poi'] = df['from_this_person_to_poi']/(df['from_messages'] + 1)
#     df['to_from'] = df['pct_from_poi']*df['pct_from_poi']
    
    #drop columns
    df = df.drop('restricted_stock_deferred', axis=1)
    df = df.drop('director_fees', axis=1)
    df = df.drop('restricted_stock', axis=1)
    
    #drop rows
    for i in df.index:
        if df.ix[i].count() < 3:
            df = df.drop(i, axis=0)
    
    df = df[df.loc[:,'salary']>0]
    
    features_list = list(df.columns)
    features_list.remove('poi')
    features = df[features_list]
    labels = df['poi']
    return features, labels

In [183]:
def get_features_labels_remove_non_salary1(df):
    '''This is where the features and labels are extracted for the 
    Udacity grader. It is also where I will do add any new variables.'''
    
    #add columns
#     df['deferred_ratio'] = df['deferred_income']/(df['total_payments'] + 1)
#     df['pct_from_poi'] = df['from_poi_to_this_person']/(df['from_messages'] + 1)
#     df['pct_to_poi'] = df['from_this_person_to_poi']/(df['from_messages'] + 1)
#     df['to_from'] = df['pct_from_poi']*df['pct_from_poi']
    
    #drop columns
    df = df.drop('restricted_stock_deferred', axis=1)
    df = df.drop('director_fees', axis=1)
    df = df.drop('restricted_stock', axis=1)
    
    #drop rows
    for i in df.index:
        if df.ix[i].count() < 3:
            df = df.drop(i, axis=0)
    
    df = df[df.loc[:,'salary']>0]
    
    features_list = list(df.columns)
    features_list.remove('poi')
    features_list.insert(0,'poi')

#     features = df[features_list]
#     labels = df['poi']
#     features = df[features_list]
#     labels = df['poi']
    return df, features_list

In [184]:
def get_features_labels_remove_non_salary_log(df):
    '''This is where the features and labels are extracted to use as arguments
    for sklearn\'s cross_validation function. It is also where I will do add
    any new variables.'''
    
    #add columns
#     df['deferred_ratio'] = df['deferred_income']/(df['total_payments'] + 1)
#     df['pct_from_poi'] = df['from_poi_to_this_person']/(df['from_messages'] + 1)
#     df['pct_to_poi'] = df['from_this_person_to_poi']/(df['from_messages'] + 1)
#     df['to_from'] = df['pct_from_poi']*df['pct_from_poi']
    
    #drop columns
    df = df.drop('restricted_stock_deferred', axis=1)
    df = df.drop('director_fees', axis=1)
    df = df.drop('restricted_stock', axis=1)
    
    #drop rows
    for i in df.index:
        if df.ix[i].count() < 3:
            df = df.drop(i, axis=0)
    
    df = df[df.loc[:,'salary']>0]
    features_list = list(df.columns)
    features_list.remove('poi')
    for i in features_list:
            df[i] = df[i] + 1
            df[i] = np.log(df[i])
   
    
    
    
    features = df[features_list]
    labels = df['poi']
    return features, labels

In [343]:
def get_features_labels_new_27(df):
    '''This is where the features and labels are extracted to use as arguments
    for sklearn\'s cross_validation function. It is also where I will do add
    any new variables.'''
    

    
    #add columns
#     df['deferred_ratio'] = df['deferred_income']/(df['total_payments'] + 1)
    df['pct_from_poi'] = df['from_poi_to_this_person']/(df['from_messages'] + 1)
    df['pct_to_poi'] = df['from_this_person_to_poi']/(df['from_messages'] + 1)
    df['to_from'] = df['pct_from_poi']*df['pct_from_poi']
    
    #drop columns
    df = df.drop('restricted_stock_deferred', axis=1)
    df = df.drop('director_fees', axis=1)
    df = df.drop('restricted_stock', axis=1)
    df = df.drop('deferral_payments', axis=1)
    df = df.drop('deferred_income', axis=1)
    
   
    #drop rows
    for i in df.index:
        if df.ix[i].count() < 3:
            df = df.drop(i, axis=0)
    
    features_list = list(df.columns)
    features_list.remove('poi')
    features = df[features_list]
    labels = df['poi']
    return features, labels

In [344]:
def get_features_labels_new_27_1(df):
    '''This is where the features and labels are extracted to use as arguments
    for sklearn\'s cross_validation function. It is also where I will do add
    any new variables.'''
    

    
    #add columns
#     df['deferred_ratio'] = df['deferred_income']/(df['total_payments'] + 1)
    df['pct_from_poi'] = df['from_poi_to_this_person']/(df['from_messages'] + 1)
    df['pct_to_poi'] = df['from_this_person_to_poi']/(df['from_messages'] + 1)
    df['to_from'] = df['pct_from_poi']*df['pct_from_poi']
    
    #drop columns
    df = df.drop('restricted_stock_deferred', axis=1)
    df = df.drop('director_fees', axis=1)
    df = df.drop('restricted_stock', axis=1)
    df = df.drop('deferral_payments', axis=1)
    df = df.drop('deferred_income', axis=1)
    
   
    #drop rows
    for i in df.index:
        if df.ix[i].count() < 3:
            df = df.drop(i, axis=0)
    
    
    features_list = list(df.columns)
    features_list.remove('poi')
    features_list.insert(0,'poi')
#     features = df[features_list]
#     labels = df['poi']
    return df, features_list

In [ ]:
def get_features_labels_new1(df):
    '''This is where the features are pared and features list is created for 
    the grader. It is also where I will do add any new variables.'''
    
    #add columns
#     df['deferred_ratio'] = df['deferred_income']/(df['total_payments'] + 1)
    df['pct_from_poi'] = df['from_poi_to_this_person']/(df['from_messages'] + 1)
    df['pct_to_poi'] = df['from_this_person_to_poi']/(df['from_messages'] + 1)
    df['to_from'] = df['pct_from_poi']*df['pct_from_poi']
    
    #drop columns
    df = df.drop('restricted_stock_deferred', axis=1)
    df = df.drop('director_fees', axis=1)
    df = df.drop('restricted_stock', axis=1)
    df = df.drop('deferral_payments', axis=1)
    df = df.drop('deferred_income', axis=1)
    
    #drop rows
    for i in df.index:
        if df.ix[i].count() < 3:
            df = df.drop(i, axis=0)
    
    features_list = list(df.columns)
    features_list.remove('poi')
    features_list.insert(0,'poi')
#     features = df[features_list]
#     labels = df['poi']
    return df, features_list

In [186]:
def get_features_labels_remove_non_salary_log1(df):
    '''This is where the features and labels are extracted for the 
    Udacity grader. It is also where I will do add any new variables.'''
    
    #add columns
#     df['deferred_ratio'] = df['deferred_income']/(df['total_payments'] + 1)
#     df['pct_from_poi'] = df['from_poi_to_this_person']/(df['from_messages'] + 1)
#     df['pct_to_poi'] = df['from_this_person_to_poi']/(df['from_messages'] + 1)
#     df['to_from'] = df['pct_from_poi']*df['pct_from_poi']
    
    #drop columns
    df = df.drop('restricted_stock_deferred', axis=1)
    df = df.drop('director_fees', axis=1)
    df = df.drop('restricted_stock', axis=1)
    
    #drop rows
    for i in df.index:
        if df.ix[i].count() < 3:
            df = df.drop(i, axis=0)
    
    df = df[df.loc[:,'salary']>0]
    

    features_list = list(df.columns)
    features_list.remove('poi')
    for i in features_list:
            df[i] = df[i] + 1
            df[i] = np.log(df[i])
   
    

    features_list.insert(0,'poi')


    return df, features_list

In [187]:
def get_features_labels_drop(df):
    '''This is where the features and labels are extracted to use as arguments
    for sklearn\'s cross_validation function. It is also where I will do add
    any new variables.
    dropping director_fees, restricted_stock_preferred'''
    df = df.drop('restricted_stock_deferred', axis=1)
    df = df.drop('director_fees', axis=1)
    df = df.drop('restricted_stock', axis=1)
    df = df.drop('total_payments', axis=1)
    df = df.drop('total_stock_value', axis=1)
    
    features_list = list(df.columns)
    features_list.remove('poi')
    features = df[features_list]
    labels = df['poi']
    return features, labels

In [188]:
def get_features_labels(df):
    '''This is where the features and labels are extracted to use as arguments
    for sklearn\'s cross_validation function. This function has no new features 
    added into the data set.'''
    features_list = list(df.columns)
    features_list.remove('poi')
    features = df[features_list]
    labels = df['poi']
    return features, labels

In [189]:
#features, labels = get_features_labels_new(df)

In [190]:
def get_train_test_split(features, labels):
    '''This gets the train test split for the sklearn runs of the model'''
    from sklearn import cross_validation
    features_train, features_test, labels_train, labels_test = cross_validation.train_test_split(features, labels, test_size=0.3, random_state=42)
    return features_train, features_test, labels_train, labels_test

In [191]:
# features_train, features_test, labels_train, labels_test = get_train_test_split(features, labels)

### Analysis Functions
The functions I will call to analyze the data.

In [192]:
# code from vivek_29420285151271 to replace f1 as scoring criterion

def precision_recall(labels,predictions):
    ind_true_pos = [i for i in range(0,len(labels)) if (predictions[i]==1) & (labels[i]==1)]
    ind_false_pos = [i for i in range(0,len(labels)) if ((predictions[i]==1) & (labels[i]==0))]
    ind_false_neg = [i for i in range(0,len(labels)) if ((predictions[i]==0) & (labels[i]==1))]
    ind_true_neg = [i for i in range(0,len(labels)) if ((predictions[i]==0) & (labels[i]==0))]
    precision = 0
    recall = 0
    
    
    ind_labels = [i for i in range(0,len(labels)) if labels[i]==1]
    
    if len(ind_labels) !=0:
        if float( len(ind_true_pos) + len(ind_false_pos))!=0:
            precision = float(len(ind_true_pos))/float( len(ind_true_pos) + len(ind_false_pos))
        if float( len(ind_true_pos) + len(ind_false_neg))!=0:
            recall = float(len(ind_true_pos))/float( len(ind_true_pos) + len(ind_false_neg))
        return precision, recall
    else:
        return -1,-1

def custom_scorer(labels, predictions):
    precision,recall = precision_recall(labels,predictions)
    min_score = min(precision, recall)
    return min_score

In [193]:
def get_outcomes(grid_object):
    '''Gets the print out of all the outcomes from the grid_search. It prints out the 
    best parameters found by the model and the outcomes of the test of the model on 
    the test set.'''
    print "Best parameters from the grid search:", pprint(gridCV_object.best_params_)
    clf_gridCV = gridCV_object.best_estimator_
    print "\nBest Estimator Accuracy:", clf_gridCV.score(features_test, labels_test)
    clf_gridCV_pred = clf_gridCV.predict(features_test)
    print "\nRecall Score:", recall_score(labels_test, clf_gridCV_pred)
    print "\nPrecision Score:", precision_score(labels_test, clf_gridCV_pred)

In [194]:
def get_outcomes_whole_data_set(grid_object):
    '''Gets the print out of all the outcomes from the grid_search. It prints out the 
    best parameters found by the model and the outcomes of the test of the model on 
    the test set, but using the whole data set rather than the train-test split.'''
    print "Best parameters from the grid search:", pprint(gridCV_object.best_params_)
    clf_gridCV = gridCV_object.best_estimator_
    print "\nBest Estimator Accuracy:", clf_gridCV.score(features_test, labels_test)
    clf_gridCV_pred = clf_gridCV.predict(features)
    print "\nRecall Score:", recall_score(labels, clf_gridCV_pred)
    print "\nPrecision Score:", precision_score(labels, clf_gridCV_pred)

### Exploratory Analysis, feature engineering

Here I am going to import the data set separately with all the variables, including serveral that I have already decided to get rid of, to do the exploratory data analysis without preconceptions. 

I call it dfE for Exploratory. I create it from the dictionary, transpose it, drop the email addresses, and change the data type to float. 

In [267]:
data = "final_project_dataset.pkl"

In [268]:
with open(data, "r") as data_file:
    data_dict = pickle.load(data_file)
dfE = pd.DataFrame(data_dict)
dfE = dfE.transpose()
dfE = dfE.drop('email_address', axis=1)
dfE = dfE.drop('TOTAL')
dfE = dfE.drop("THE TRAVEL AGENCY IN THE PARK")
dfE = dfE.drop("loan_advances", axis=1)
dfE = dfE.astype(float)

In [269]:
#number of features about each person
len(dfE.ix[0,:])

19

In [270]:
#number of poi's 
len(dfE.ix[dfE['poi']==1])
# why doesn't the second dfE need an .ix? 

18

In [271]:
#total value of stock belonging to James Prentice
#dfE.ix[:,0]
# how many people have no salary (count ignores null values)
# dfE['salary'].count()
# percentage of people with NaN for total payments
# ntNaN = dfE['total_payments'].count()
# n = float(len(dfE))
# NaN = (n-ntNaN)
# (NaN/n)*100.0
# percentage of poi's with NaN for total_payments
#dfE.groupby('poi')['total_payments'].count()
dfE.isnull().sum()

bonus                         63
deferral_payments            106
deferred_income               96
director_fees                128
exercised_stock_options       43
expenses                      50
from_messages                 58
from_poi_to_this_person       58
from_this_person_to_poi       58
long_term_incentive           79
other                         53
poi                            0
restricted_stock              35
restricted_stock_deferred    127
salary                        50
shared_receipt_with_poi       58
to_messages                   58
total_payments                21
total_stock_value             19
dtype: int64

There is just so little data that I am hesitant to cut out anything. Still, when over 100 cases are missing for a particular variable it seems like it is almost surely doing more harm than good, so I am going to take out the variables with more than 90 cases missing. 

In [272]:
for i in dfE.columns: 
    if dfE[i].isnull().sum() > 90:
        print i

deferral_payments
deferred_income
director_fees
restricted_stock_deferred


In [266]:
dfE.isnull().sum()

bonus                      63
exercised_stock_options    43
expenses                   50
from_messages              58
from_poi_to_this_person    58
from_this_person_to_poi    58
long_term_incentive        79
other                      53
poi                         0
restricted_stock           35
salary                     50
shared_receipt_with_poi    58
to_messages                58
total_payments             21
total_stock_value          19
dtype: int64

#### Big correlation analysis

In [ ]:
dfE.corr()

I am really hesitant to throw out any variables with such a small data set, but the variables for which we can't even compute a correlation with the dependent variable seem like they can't be helping anything and must be increasing the dimensionality of the data set. So I will drop restricted_stock_deferred and director_fees. Also, restricted_stock does not seem to be helping much, either, since it has a 0.000107 correlation with the dependent variable. 

#### Get a list of how many missing variables each column has.

In [ ]:
dfE.count(axis=0)

In [ ]:
dfE.describe()

It appears that some variables occur only once or a few times. This makes them rather unreliable. They may have a correlation with the dependent variable but it would only be an accident. Or they might correlate in this data set but not in other data. So, especially with the models that are subject to the cures of dimensionality like SVC. So, I will also drop loan_advances (4 instances), total_payments and total_stock_value (both with only 1 instance). UPDATE: I misread the counts for the last two variables. It was a + e02. They stay in.  

#### Get a list of how many missing variables there are in each row

In [ ]:
for i in dfE.index:
    if dfE.ix[i].count() < 4:
        dfE = dfE.drop(i, axis=0)

In [ ]:
dfE_salary = dfE[dfE.loc[:,'salary']>0]

In [ ]:
len(dfE_salary[dfE.loc[:,'poi']==1])

Ok, so by getting rid of the people with non-salary information we only loose 2 poi's. Worth it? Since there are only 2 removing them only increases or decreases precision. 

In [ ]:
dfE.groupby('poi')['restricted_stock_deferred'].agg(np.median).plot(kind='barh')

In [ ]:
df.groupby('poi')['director_fees'].agg(np.median).plot(kind='barh')

So one thing I am going to do with this information is to remove the variables that are not associated in any way with the poi's. That might at least deal with the 1.0's problem. 

How many people are in the data set? 

In [ ]:
len(dfE.ix[:,0])

Ok, so I dropped the people in the data set for whom I had three or fewer pieces of information and it decreased the performance of every model. So, they are staying in. The people dropped by the standard of having three or fewer pieces of information were not poi's so I didn't think it would make that much difference, but it did. 

### Models

### Random Forests

Random forests are prone to over fitting and so are also something of a problem when dealing with a small data set. Also, very high dimensional data can be a problem. [Jake VanderPlas](https://www.youtube.com/watch?v=L7R4HUQ-eQ0)

In [205]:
df = import_data(data)

#df.fillna(inplace=True, value=0)
features, labels = get_features_labels_remove_non_salary(df)
features_train, features_test, labels_train, labels_test = get_train_test_split(features, labels)

#deal with missing data

imp = Imputer(strategy='mean')
features_train = imp.fit_transform(features_train)
features_test = imp.fit_transform(features_test)

from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(max_depth=6 , max_features=10 , n_estimators=1000)
clf.fit(features_train, labels_train)
pred = clf.predict(features_test)

#caculate accuracy
from sklearn.metrics import accuracy_score
print accuracy_score(pred, labels_test)

print "\nRecall Score:", recall_score(labels_test, pred)
print "\nPrecision Score:", precision_score(labels_test, pred)


0.689655172414

Recall Score: 0.142857142857

Precision Score: 0.25


In [340]:
### Gradiant Boosting Classifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.metrics import make_scorer
from sklearn.ensemble import GradientBoostingClassifier

# Get custom scorer
score = make_scorer(custom_scorer, greater_is_better=True)

# get the df
df = import_data(data)

# impute 0 to missing data values
#df.fillna(inplace=True, value=0) 

# Get data, here with the features unrealated to poi dropped AND Tanya's features added.
features, labels = get_features_labels_new_27(df)

# Get the test-train split
features_train, features_test, labels_train, labels_test = cross_validation.train_test_split(features, labels)

from sklearn.preprocessing import Imputer
imp = Imputer(missing_values='NaN', strategy='mean', axis=0)
features_train = imp.fit_transform(features_train)
features_test = imp.fit_transform(features_test)

from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
features_train = scaler.fit_transform(features_train)
features_test = scaler.fit_transform(features_test)

# Build pipeline
Pipeline = Pipeline([
#        ('imp', Imputer(missing_values='NaN')),
        ('pca', PCA()),
        ('clf', GradientBoostingClassifier(n_estimators=100))
    ])

# Build Grid
x = [x for x in range(2,7)]
f = [f for f in range(3,len(features.columns))]
param_grid = { #'imp__strategy': ['most_frequent', 'median', 'mean'],
               'pca__n_components': x,
               'clf__learning_rate': [0.5, 0.7, 0.8, 0.9],
               'clf__max_features': [0.05, 0.1, 0.5, .7, 1],
               'clf__max_depth': [3, 4, 5, 6, 7, 8]}

# set model parameters to grid search object
gridCV_object = GridSearchCV(estimator = Pipeline, 
                             param_grid = param_grid,
                             scoring = 'f1',
                             cv = StratifiedShuffleSplit(labels_train, test_size=0.1,  n_iter=10))

# train the model
gridCV_object.fit(features_train, labels_train)

print "GBC with email variables added in and the five 90+ NaN columns droped"
# apply model to test data, print results
get_outcomes(gridCV_object)

GBC with email variables added in and the five 90+ NaN columns droped
Best parameters from the grid search:{'clf__learning_rate': 0.7,
 'clf__max_depth': 6,
 'clf__max_features': 0.1,
 'pca__n_components': 4}
 None

Best Estimator Accuracy: 0.8

Recall Score: 0.5

Precision Score: 0.285714285714


I am going to start keeping records on the results of the gradient boosting classifier as I make changes to the data set. 

Here the GradientBoostingClassier had n_estimators=100 and used 0's for the missing data. The data set was `get_features_labels_new_27()` and had all the columns with more than 90 NaN's (about 4) removed. 

```

Best parameters from the grid search:{'clf__learning_rate': 0.05, 'clf__max_depth': 4, 'clf__max_features': 0.7}
 None

Best Estimator Accuracy: 0.888888888889

Recall Score: 0.0

Precision Score: 0.0
```
Not very promising. 

### Adding Email Data

GBC with email variables added in and the five 90+ NaN columns droped. Also used the grid search for imputation method. 

```
Best parameters from the grid search:{'clf__learning_rate': 0.05,
 'clf__max_depth': 4,
 'clf__max_features': 0.5,
 'imp__strategy': 'mean'}
 None

Best Estimator Accuracy: 0.8

Recall Score: 0.142857142857

Precision Score: 0.5
```
Just can't figure out how to get the recall up! 

### MinMaxScaling before Standardization
GBC with email variables added in and the five 90+ NaN columns droped

Did MinMax Scaling and imputing mean values for missing data before doing grid search. Only did 10 iternations. Going to try 1000
```
Best parameters from the grid search:{'clf__learning_rate': 0.1, 'clf__max_depth': 3, 'clf__max_features': 0.5}
 None

Best Estimator Accuracy: 0.828571428571

Recall Score: 0.333333333333

Precision Score: 0.5
```

### With 100 iterations
Somehow it gets worse on precision while getting better on Recall. There might be something wrong with the score function: 

```
GBC with email variables added in and the five 90+ NaN columns droped
Best parameters from the grid search:{'clf__learning_rate': 0.5, 'clf__max_depth': 4, 'clf__max_features': 0.5}
 None

Best Estimator Accuracy: 0.685714285714

Recall Score: 0.5

Precision Score: 0.0909090909091
```
Maybe there is something wrong with the score function I have borrowed so I will try the f1:
### With f1 as scoring function


GBC with email variables added in and the five 90+ NaN columns droped and MinMaxScaling done before hand. 
Also, 100 iterations.
```
Best parameters from the grid search:{'clf__learning_rate': 0.05, 'clf__max_depth': 3, 'clf__max_features': 1}
 None

Best Estimator Accuracy: 0.828571428571

Recall Score: 0.25

Precision Score: 0.25
```
I am going to stick with the f1 scorer for a while now. Next I will try the PCA before giving it GBC.

### With PCA in pipeline
Got a huge increase in Recall but precision is bad again. 

```
GBC with email variables added in and the five 90+ NaN columns droped
Best parameters from the grid search:{'clf__learning_rate': 0.05,
 'clf__max_depth': 4,
 'clf__max_features': 0.7,
 'pca__n_components': 3}
 None

Best Estimator Accuracy: 0.542857142857

Recall Score: 0.6

Precision Score: 0.176470588235
```
Just to keep ourselves sane we will try it with the custom score function. 



### w/ Custom Scorer

```
GBC with email variables added in and the five 90+ NaN columns droped
Best parameters from the grid search:{'clf__learning_rate': 0.1,
 'clf__max_depth': 4,
 'clf__max_features': 0.7,
 'pca__n_components': 10}
 None

Best Estimator Accuracy: 0.685714285714

Recall Score: 0.2

Precision Score: 0.125
```
Ok, so the custom scorer sucks. No more of that. I will try the f1 scorer with 100 iterations, though. 

### F1 with 100 iterations
Gave up. Took too long
Now adding parameters. 

```
GBC with email variables added in and the five 90+ NaN columns droped
Best parameters from the grid search:{'clf__learning_rate': 0.7,
 'clf__max_depth': 6,
 'clf__max_features': 0.1,
 'pca__n_components': 4}
 None

Best Estimator Accuracy: 0.8

Recall Score: 0.5

Precision Score: 0.285714285714
```
That was pretty good with only 10 iterations. The learning rate was the highest number in the list so there might still be room or improvement. 

### Adding higher learning rate choice
Trimming the pca and learning rate numbers of alternatives that were rejected. 

### Naive Bayes

In [329]:
df = import_data(data)

#df.fillna(inplace=True, value=0)
features, labels = get_features_labels_new_27(df)
features_train, features_test, labels_train, labels_test = get_train_test_split(features, labels)

#deal with missing data
from sklearn.svm import LinearSVC
imp = Imputer(strategy='most_frequent')
features_train = imp.fit_transform(features_train)
features_test = imp.fit_transform(features_test)

from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
features_train = scaler.fit_transform(features_train)
features_test = scaler.fit_transform(features_test)

from sklearn.decomposition import PCA
pca = PCA(n_components=3)
features_train = pca.fit_transform(features_train)
features_test = pca.fit_transform(features_test)


from sklearn.naive_bayes import GaussianNB
clf = GaussianNB()
clf.fit(features_train, labels_train)
pred = clf.predict(features_test)

#caculate accuracy
from sklearn.metrics import accuracy_score
print accuracy_score(pred, labels_test)

print "\nRecall Score:", recall_score(labels_test, pred)
print "\nPrecision Score:", precision_score(labels_test, pred)

0.880952380952

Recall Score: 0.25

Precision Score: 0.333333333333


In [330]:
#features

### LinearSVC

In [ ]:
df = import_data(data)

# impute 0 to missing data values
# df.fillna(inplace=True, value=0) 
features, labels = get_features_labels_remove_non_salary(df)
features_train, features_test, labels_train, labels_test = get_train_test_split(features, labels)

#deal with missing data
from sklearn.svm import LinearSVC
imp = Imputer(strategy='most_frequent')
features_train = imp.fit_transform(features_train)
features_test = imp.fit_transform(features_test)

#LinearSVC without pipeline
from sklearn.preprocessing import Imputer
clf = LinearSVC()
clf.fit(features_train, labels_train)
pred = clf.predict(features_test)

#caculate accuracy
from sklearn.metrics import accuracy_score
print accuracy_score(pred, labels_test)

print "\nRecall Score:", recall_score(labels_test, pred)
print "\nPrecision Score:", precision_score(labels_test, pred)

## SVC

In [347]:
#import data
df = import_data(data)
# df.fillna(inplace=True, value=0)
features, labels = get_features_labels_new_27(df)
features_train, features_test, labels_train, labels_test = get_train_test_split(features, labels)
# print "after split: %r\n" %features_train[1:2]

#deal with missing data
from sklearn.preprocessing import Imputer
imp = Imputer(missing_values='NaN', strategy="most_frequent")
features_train = imp.fit_transform(features_train)
features_test = imp.fit_transform(features_test)
# print "after imputer: %r\n" %features_train[1:2]

#Feature Scaling
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
features_train = scaler.fit_transform(features_train)
features_test = scaler.fit_transform(features_test)
# print "after scaling: %r\n" %features_train[1:2]

#PCA
from sklearn.decomposition import PCA
pca = PCA(n_components=9)
features_train = pca.fit_transform(features_train)
features_test = pca.fit_transform(features_test)
# print "after scaling: %r\n" %features_train[1:2]
print "PCA explained Variance: %r\n" %pca.explained_variance_ratio_

#LinearSVC without pipeline
from sklearn.svm import SVC
clf = SVC(kernel='rbf', C=10000, gamma=0.01)
clf.fit(features_train, labels_train)
pred = clf.predict(features_test)

# caculate accuracy
from sklearn.metrics import accuracy_score
print accuracy_score(pred, labels_test)

print "\nRecall Score:", recall_score(labels_test, pred)
print "\nPrecision Score:", precision_score(labels_test, pred)

PCA explained Variance: array([ 0.44725245,  0.17013664,  0.12259992,  0.07351105,  0.05803404,
        0.04368933,  0.02719228,  0.01921261,  0.01375639])

0.833333333333

Recall Score: 0.5

Precision Score: 0.285714285714


SVC results: Well, the linear kernel just won't work. There is obviously no linear separating plane in this data. The only kernels that finish in a reasonable amount of time are the sigmoid and rbf kernels. The gamma and C parameters don't seem to have any effect on the accuracy of the model. 

I looked at the predictions and I think the problem is that the model predicts that no one is a poi. This gives about 88% accuracy because 88% of the people in the test set are not poi's. In fact, when I test this by running the code: 
```
1-(5/float(len(labels_test)))
```
I get the exact same number (0.886363636364) as I was getting from the accuracy_score. The recall and accuracy just stay at 0.0 for both. It just doesn't care!

UPDATE: got it to perform great on the SVC here but it did lousey on the grader. With pca=9, C=10000000, gamma = 0.01 with the new columns outIt gave me like 
```
0.904761904762

Recall Score: 0.8

Precision Score: 0.571428571429
```
But was an utter fail on the grader. Both precision and recall under 0.3. It is starting to feel like this is all bullshit! Why even bother if you get completely different results on one measure compared to the other. 

In [382]:
print range(1,4)

[1, 2, 3]


In [387]:
# SVC rbf
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.metrics import make_scorer
from sklearn.feature_selection import SelectKBest

# Get custom scorer
score = make_scorer(custom_scorer, greater_is_better=True)

# get the df
df = import_data(data)

# Get data, here with the features unrealated to poi dropped AND Tanya's features added.
features, labels = get_features_labels_new_27(df)

# Get the test-train split
#features_train, features_test, labels_train, labels_test = cross_validation.train_test_split(features, labels)

# Build pipeline
Pipeline = Pipeline([
        ('imp', Imputer(missing_values='NaN', strategy='median')),
        ('std', MinMaxScaler()),
        ('selection', SelectKBest()),
        ('pca', PCA()),
        ('clf', GradientBoostingClassifier(random_state=0, loss="exponential", n_estimators=100))
    ])

# Build Grid
k = [k for k in range(15,17)]
x = [x for x in range(2,5)]
# c = [1000000,100000,10000,100,10,0.01]
# g = [0.1,0.01,0.001,0.0001]
d = [d for d in range(8,13)]
#e = [50, 100]
r = [0.3, 0.5, 0.6, 0.7]
#l = ["deviance", "exponential"]
param_grid = {'selection__k': k,
              'pca__n_components': x,
              #'imp__strategy': ['most_frequent', 'mean', 'median'],
              #'clf__n_estimators': e,
              'clf__learning_rate': r,
              'clf__max_depth': d#,
              #'clf__loss': l
             }

# set model parameters to grid search object
gridCV_object = GridSearchCV(estimator = Pipeline, 
                             param_grid = param_grid,
                             scoring = score,
                             cv = StratifiedShuffleSplit(labels, test_size=0.1,  n_iter=10))

# train the model
gridCV_object.fit(features, labels)

# apply model to test data, print results
get_outcomes(gridCV_object)

#print gridCV_object.feature_importances_

Best parameters from the grid search:{'clf__learning_rate': 0.6,
 'clf__max_depth': 9,
 'pca__n_components': 3,
 'selection__k': 16}
 None

Best Estimator Accuracy: 1.0

Recall Score: 1.0

Precision Score: 1.0


### Same model submitted to the grader: 

Start from begining

In [354]:
#import data
#import data
df = import_data(data)
df, features_list = get_features_labels_new_27_1(df)
# print "after split: %r\n" %features_train[1:2]



#change back to dictionary
df1 = df.transpose()
df1 = df1.to_dict()

In [399]:
from tester import test_classifier
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
from sklearn.decomposition import PCA
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.preprocessing import MinMaxScaler
# clf = RandomForestClassifier(max_depth=7 , max_features=10 , n_estimators=1000)

clf = Pipeline([
        ('imp', Imputer(missing_values='NaN', strategy='median')),
        ('scaler', MinMaxScaler()),
        ('selection', SelectKBest(k=15)),
        ('pca', PCA(n_components=3)),
        ('clf', GradientBoostingClassifier(n_estimators=100,loss="exponential", max_depth=9, learning_rate=0.7))
    ])

print "\n\nAnd these are the results going through the test classifier:\n"
test_classifier(clf, df1, features_list, folds = 1000)



And these are the results going through the test classifier:

Pipeline(steps=[('imp', Imputer(axis=0, copy=True, missing_values='NaN', strategy='median', verbose=0)), ('scaler', MinMaxScaler(copy=True, feature_range=(0, 1))), ('selection', SelectKBest(k=15, score_func=<function f_classif at 0x10e2f40c8>)), ('pca', PCA(copy=True, n_components=3, whiten=False)), ('clf', ...      presort='auto', random_state=None, subsample=1.0, verbose=0,
              warm_start=False))])
	Accuracy: 0.82329	Precision: 0.37092	Recall: 0.34050	F1: 0.35506	F2: 0.34618
	Total predictions: 14000	True positives:  681	False positives: 1155	False negatives: 1319	True negatives: 10845



## Grader Results

### SVC
Here are some results from the grader: 
```

And these are the results going through the test classifier:

Pipeline(steps=[('imp', Imputer(axis=0, copy=True, missing_values='NaN', strategy='most_frequent',
    verbose=0)), ('scaler', MinMaxScaler(copy=True, feature_range=(0, 1))), ('pca', PCA(copy=True, n_components=9, whiten=False)), ('clf', SVC(C=10000, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma=0.1, kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False))])
	Accuracy: 0.81729	Precision: 0.31645	Recall: 0.24050	F1: 0.27330	F2: 0.25263
	Total predictions: 14000	True positives:  481	False positives: 1039	False negatives: 1519	True negatives: 10961
```
The thing is that the precision was there but the recall was not. Those parameters came from running the SVC with f1 scoring with a training-test split. It worked well in with the StratifiedShuffleSplit tester with 100 iterations but did not break the 0.3 thresholds for both with the Udacity Grader. So now I am going to try new parameters found by running the StratifiedShuffleSplit test with the full data set.  

```



And these are the results going through the test classifier:

Pipeline(steps=[('imp', Imputer(axis=0, copy=True, missing_values='NaN', strategy='most_frequent',
    verbose=0)), ('scaler', MinMaxScaler(copy=True, feature_range=(0, 1))), ('pca', PCA(copy=True, n_components=6, whiten=False)), ('clf', SVC(C=1000000, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma=0.1, kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False))])
	Accuracy: 0.76743	Precision: 0.20572	Recall: 0.21950	F1: 0.21239	F2: 0.21660
	Total predictions: 14000	True positives:  439	False positives: 1695	False negatives: 1561	True negatives: 10305
```
So these results are even worse. The lesson is that using the whole data set gives even more misleading parameters for the grader. 


### Select K Best
So here is what is driving me crazy. I get this result from using select K Best with the SVC: 
```
Best parameters from the grid search:{'selection__k': 13}
 None

Best Estimator Accuracy: 0.971428571429

Recall Score: 0.833333333333

Precision Score: 1.0
```
Great, heh? But when I give it to the grader: 
```
And these are the results going through the test classifier:

Pipeline(steps=[('imp', Imputer(axis=0, copy=True, missing_values='NaN', strategy='most_frequent',
    verbose=0)), ('scaler', MinMaxScaler(copy=True, feature_range=(0, 1))), ('selection', SelectKBest(k=13, score_func=<function f_classif at 0x10e2f40c8>)), ('pca', PCA(copy=True, n_components=6, whiten=False)...,
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False))])
	Accuracy: 0.76779	Precision: 0.20172	Recall: 0.21150	F1: 0.20649	F2: 0.20947
	Total predictions: 14000	True positives:  423	False positives: 1674	False negatives: 1577	True negatives: 10326
```

How am I going to make this better? 

### Winner? 


And these are the results going through the test classifier:
```
Pipeline(steps=[('imp', Imputer(axis=0, copy=True, missing_values='NaN', strategy='most_frequent',
    verbose=0)), ('scaler', MinMaxScaler(copy=True, feature_range=(0, 1))), ('selection', SelectKBest(k=16, score_func=<function f_classif at 0x10e2f40c8>)), ('pca', PCA(copy=True, n_components=3, whiten=False)...      presort='auto', random_state=None, subsample=1.0, verbose=0,
              warm_start=False))])
	Accuracy: 0.82979	Precision: 0.37872	Recall: 0.29900	F1: 0.33417	F2: 0.31214
	Total predictions: 14000	True positives:  598	False positives:  981	False negatives: 1402	True negatives: 11019
```
Close enough for government work I would say. 

One more test

## Victory!

And these are the results going through the test classifier:
```
Pipeline(steps=[('imp', Imputer(axis=0, copy=True, missing_values='NaN', strategy='mean', verbose=0)), ('scaler', MinMaxScaler(copy=True, feature_range=(0, 1))), ('selection', SelectKBest(k=16, score_func=<function f_classif at 0x10e2f40c8>)), ('pca', PCA(copy=True, n_components=3, whiten=False)), ('clf', Gr...      presort='auto', random_state=None, subsample=1.0, verbose=0,
              warm_start=False))])
	Accuracy: 0.81564	Precision: 0.33924	Recall: 0.30650	F1: 0.32204	F2: 0.31253
	Total predictions: 14000	True positives:  613	False positives: 1194	False negatives: 1387	True negatives: 10806
```    
### Again, but with the custom scoring function



And these are the results going through the test classifier:

```
Pipeline(steps=[('imp', Imputer(axis=0, copy=True, missing_values='NaN', strategy='median', verbose=0)), ('scaler', MinMaxScaler(copy=True, feature_range=(0, 1))), ('selection', SelectKBest(k=16, score_func=<function f_classif at 0x10e2f40c8>)), ('pca', PCA(copy=True, n_components=3, whiten=False)), ('clf', ...      presort='auto', random_state=None, subsample=1.0, verbose=0,
              warm_start=False))])
	Accuracy: 0.82664	Precision: 0.38757	Recall: 0.36800	F1: 0.37753	F2: 0.37175
	Total predictions: 14000	True positives:  736	False positives: 1163	False negatives: 1264	True negatives: 10837
```
So now I finally have something that works well. The thing is that the function gives me 1.0 on all the parameters so the answers at this point from the tests are just arbitrary? Maybe there are lots of parameters that give me 1.0 and it is just stopping at the first one that gives me that? 

### more

And these are the results going through the test classifier:
```
Pipeline(steps=[('imp', Imputer(axis=0, copy=True, missing_values='NaN', strategy='median', verbose=0)), ('scaler', MinMaxScaler(copy=True, feature_range=(0, 1))), ('selection', SelectKBest(k=16, score_func=<function f_classif at 0x10e2f40c8>)), ('pca', PCA(copy=True, n_components=3, whiten=False)), ('clf', ...      presort='auto', random_state=None, subsample=1.0, verbose=0,
              warm_start=False))])
	Accuracy: 0.82586	Precision: 0.38510	Recall: 0.36700	F1: 0.37583	F2: 0.37048
	Total predictions: 14000	True positives:  734	False positives: 1172	False negatives: 1266	True negatives: 10828
```   
## best?

And these are the results going through the test classifier:

```
Pipeline(steps=[('imp', Imputer(axis=0, copy=True, missing_values='NaN', strategy='median', verbose=0)), ('scaler', MinMaxScaler(copy=True, feature_range=(0, 1))), ('selection', SelectKBest(k=16, score_func=<function f_classif at 0x10e2f40c8>)), ('pca', PCA(copy=True, n_components=3, whiten=False)), ('clf', ...      presort='auto', random_state=None, subsample=1.0, verbose=0,
              warm_start=False))])
	Accuracy: 0.82450	Precision: 0.38203	Recall: 0.37000	F1: 0.37592	F2: 0.37235
	Total predictions: 14000	True positives:  740	False positives: 1197	False negatives: 1260	True negatives: 10803
```
### Diminishing returns? 
Tried moving the learning rate up to 0.8 from 0.7. 

```

And these are the results going through the test classifier:

Pipeline(steps=[('imp', Imputer(axis=0, copy=True, missing_values='NaN', strategy='median', verbose=0)), ('scaler', MinMaxScaler(copy=True, feature_range=(0, 1))), ('selection', SelectKBest(k=16, score_func=<function f_classif at 0x10e2f40c8>)), ('pca', PCA(copy=True, n_components=3, whiten=False)), ('clf', ...      presort='auto', random_state=None, subsample=1.0, verbose=0,
              warm_start=False))])
	Accuracy: 0.82464	Precision: 0.38108	Recall: 0.36450	F1: 0.37260	F2: 0.36770
	Total predictions: 14000	True positives:  729	False positives: 1184	False negatives: 1271	True negatives: 10816
```
### Not much better. Learning rate 0.6
```

And these are the results going through the test classifier:

Pipeline(steps=[('imp', Imputer(axis=0, copy=True, missing_values='NaN', strategy='median', verbose=0)), ('scaler', MinMaxScaler(copy=True, feature_range=(0, 1))), ('selection', SelectKBest(k=16, score_func=<function f_classif at 0x10e2f40c8>)), ('pca', PCA(copy=True, n_components=3, whiten=False)), ('clf', ...      presort='auto', random_state=None, subsample=1.0, verbose=0,
              warm_start=False))])
	Accuracy: 0.82543	Precision: 0.38279	Recall: 0.36250	F1: 0.37237	F2: 0.36638
	Total predictions: 14000	True positives:  725	False positives: 1169	False negatives: 1275	True negatives: 10831
```

## Exponential Wins!
The best score yet. Maybe the thing was getting 1's on everything so it was just getting me the first thing that showed up? 

```



And these are the results going through the test classifier:

Pipeline(steps=[('imp', Imputer(axis=0, copy=True, missing_values='NaN', strategy='median', verbose=0)), ('scaler', MinMaxScaler(copy=True, feature_range=(0, 1))), ('selection', SelectKBest(k=16, score_func=<function f_classif at 0x10e2f40c8>)), ('pca', PCA(copy=True, n_components=3, whiten=False)), ('clf', ...      presort='auto', random_state=None, subsample=1.0, verbose=0,
              warm_start=False))])
	Accuracy: 0.82864	Precision: 0.39394	Recall: 0.37050	F1: 0.38186	F2: 0.37496
	Total predictions: 14000	True positives:  741	False positives: 1140	False negatives: 1259	True negatives: 10860
```

### Going Down, reduced select K best to 15

```

And these are the results going through the test classifier:

Pipeline(steps=[('imp', Imputer(axis=0, copy=True, missing_values='NaN', strategy='median', verbose=0)), ('scaler', MinMaxScaler(copy=True, feature_range=(0, 1))), ('selection', SelectKBest(k=15, score_func=<function f_classif at 0x10e2f40c8>)), ('pca', PCA(copy=True, n_components=3, whiten=False)), ('clf', ...      presort='auto', random_state=None, subsample=1.0, verbose=0,
              warm_start=False))])
	Accuracy: 0.82250	Precision: 0.36668	Recall: 0.33350	F1: 0.34931	F2: 0.33965
	Total predictions: 14000	True positives:  667	False positives: 1152	False negatives: 1333	True negatives: 10848
```

### Max Depth up to 10 doesn't help

And these are the results going through the test classifier:
```
Pipeline(steps=[('imp', Imputer(axis=0, copy=True, missing_values='NaN', strategy='median', verbose=0)), ('scaler', MinMaxScaler(copy=True, feature_range=(0, 1))), ('selection', SelectKBest(k=16, score_func=<function f_classif at 0x10e2f40c8>)), ('pca', PCA(copy=True, n_components=3, whiten=False)), ('clf', ...      presort='auto', random_state=None, subsample=1.0, verbose=0,
              warm_start=False))])
	Accuracy: 0.82643	Precision: 0.38660	Recall: 0.36650	F1: 0.37628	F2: 0.37035
	Total predictions: 14000	True positives:  733	False positives: 1163	False negatives: 1267	True negatives: 10837
```

## Highest Precision from highest learning rate, 0.8

```


And these are the results going through the test classifier:

Pipeline(steps=[('imp', Imputer(axis=0, copy=True, missing_values='NaN', strategy='median', verbose=0)), ('scaler', MinMaxScaler(copy=True, feature_range=(0, 1))), ('selection', SelectKBest(k=16, score_func=<function f_classif at 0x10e2f40c8>)), ('pca', PCA(copy=True, n_components=3, whiten=False)), ('clf', ...      presort='auto', random_state=None, subsample=1.0, verbose=0,
              warm_start=False))])
	Accuracy: 0.82800	Precision: 0.39137	Recall: 0.36750	F1: 0.37906	F2: 0.37204
	Total predictions: 14000	True positives:  735	False positives: 1143	False negatives: 1265	True negatives: 10857
```

### Learning Rate at 0.9 no better
```

And these are the results going through the test classifier:

Pipeline(steps=[('imp', Imputer(axis=0, copy=True, missing_values='NaN', strategy='median', verbose=0)), ('scaler', MinMaxScaler(copy=True, feature_range=(0, 1))), ('selection', SelectKBest(k=16, score_func=<function f_classif at 0x10e2f40c8>)), ('pca', PCA(copy=True, n_components=3, whiten=False)), ('clf', ...      presort='auto', random_state=None, subsample=1.0, verbose=0,
              warm_start=False))])
	Accuracy: 0.82629	Precision: 0.38387	Recall: 0.35700	F1: 0.36995	F2: 0.36207
	Total predictions: 14000	True positives:  714	False positives: 1146	False negatives: 1286	True negatives: 10854
```
### Trying Whiten=True
```
Pipeline(steps=[('imp', Imputer(axis=0, copy=True, missing_values='NaN', strategy='median', verbose=0)), ('scaler', MinMaxScaler(copy=True, feature_range=(0, 1))), ('selection', SelectKBest(k=16, score_func=<function f_classif at 0x10e2f40c8>)), ('pca', PCA(copy=True, n_components=3, whiten=True)), ('clf', G...      presort='auto', random_state=None, subsample=1.0, verbose=0,
              warm_start=False))])
	Accuracy: 0.82479	Precision: 0.38035	Recall: 0.36000	F1: 0.36989	F2: 0.36389
	Total predictions: 14000	True positives:  720	False positives: 1173	False negatives: 1280	True negatives: 10827
```
### Tried Deviance 
```
Pipeline(steps=[('imp', Imputer(axis=0, copy=True, missing_values='NaN', strategy='median', verbose=0)), ('scaler', MinMaxScaler(copy=True, feature_range=(0, 1))), ('selection', SelectKBest(k=16, score_func=<function f_classif at 0x10e2f40c8>)), ('pca', PCA(copy=True, n_components=3, whiten=True)), ('clf', G...      presort='auto', random_state=None, subsample=1.0, verbose=0,
              warm_start=False))])
	Accuracy: 0.82421	Precision: 0.37989	Recall: 0.36450	F1: 0.37203	F2: 0.36748
	Total predictions: 14000	True positives:  729	False positives: 1190	False negatives: 1271	True negatives: 10810
```

Seems like going to back to exponential is the right idea.

###   GradientBoostingClassifier(n_estimators=100,loss="exponential", max_depth=8, learning_rate=0.8)
```

Pipeline(steps=[('imp', Imputer(axis=0, copy=True, missing_values='NaN', strategy='median', verbose=0)), ('scaler', MinMaxScaler(copy=True, feature_range=(0, 1))), ('selection', SelectKBest(k=16, score_func=<function f_classif at 0x10e2f40c8>)), ('pca', PCA(copy=True, n_components=3, whiten=True)), ('clf', G...      presort='auto', random_state=None, subsample=1.0, verbose=0,
              warm_start=False))])
	Accuracy: 0.82657	Precision: 0.38593	Recall: 0.36200	F1: 0.37358	F2: 0.36655
	Total predictions: 14000	True positives:  724	False positives: 1152	False negatives: 1276	True negatives: 10848
```
### 'n_estimators=100,loss="exponential", max_depth=9, learning_rate=0.7
whiten was false
```
Pipeline(steps=[('imp', Imputer(axis=0, copy=True, missing_values='NaN', strategy='median', verbose=0)), ('scaler', MinMaxScaler(copy=True, feature_range=(0, 1))), ('selection', SelectKBest(k=16, score_func=<function f_classif at 0x10e2f40c8>)), ('pca', PCA(copy=True, n_components=3, whiten=False)), ('clf', ...      presort='auto', random_state=None, subsample=1.0, verbose=0,
              warm_start=False))])
	Accuracy: 0.82514	Precision: 0.38272	Recall: 0.36550	F1: 0.37391	F2: 0.36882
	Total predictions: 14000	True positives:  731	False positives: 1179	False negatives: 1269	True negatives: 10821
```
### kbest = 16, PCA(n_components=4) n_estimators=100, loss="exponential", max_depth=9, learning_rate=0.7


```
Pipeline(steps=[('imp', Imputer(axis=0, copy=True, missing_values='NaN', strategy='median', verbose=0)), ('scaler', MinMaxScaler(copy=True, feature_range=(0, 1))), ('selection', SelectKBest(k=16, score_func=<function f_classif at 0x10e2f40c8>)), ('pca', PCA(copy=True, n_components=4, whiten=False)), ('clf', ...      presort='auto', random_state=None, subsample=1.0, verbose=0,
              warm_start=False))])
	Accuracy: 0.81100	Precision: 0.32910	Recall: 0.31100	F1: 0.31979	F2: 0.31446
	Total predictions: 14000	True positives:  622	False positives: 1268	False negatives: 1378	True negatives: 10732
```

### , SelectKBest(k=15)),PCA(n_components=3) n_estimators=100,loss="exponential", max_depth=9, learning_rate=0.7))

```
Pipeline(steps=[('imp', Imputer(axis=0, copy=True, missing_values='NaN', strategy='median', verbose=0)), ('scaler', MinMaxScaler(copy=True, feature_range=(0, 1))), ('selection', SelectKBest(k=15, score_func=<function f_classif at 0x10e2f40c8>)), ('pca', PCA(copy=True, n_components=3, whiten=False)), ('clf', ...      presort='auto', random_state=None, subsample=1.0, verbose=0,
              warm_start=False))])
	Accuracy: 0.82329	Precision: 0.37092	Recall: 0.34050	F1: 0.35506	F2: 0.34618
	Total predictions: 14000	True positives:  681	False positives: 1155	False negatives: 1319	True negatives: 10845
```
I think I have enough to go on now. 

## Results
First attempt with the new scorer. Not sure whether this is the old or the new data set. 
```

Best parameters from the grid search:{'clf__max_depth': 8, 'imp__strategy': 'median', 'pca__n_components': 9}
 None

Best Estimator Accuracy: 0.772727272727


Recall Score: 0.2


Precision Score: 0.142857142857
```
This time I am doing it with the original data set for sure. 
```

Best parameters from the grid search:{'clf__max_depth': 5,
 'imp__strategy': 'most_frequent',
 'pca__n_components': 10}
 None

Best Estimator Accuracy: 0.886363636364

Recall Score: 0.2

Precision Score: 0.5
```
With the original data in the data set we seem to do a little better than with the new, created variables. Precision seems to be a lot better with the original, untreated data. 
 
I am running it one more time with \_new data just to make sure that was the problem. 
```
Best parameters from the grid search:{'clf__max_depth': 9, 'imp__strategy': 'median', 'pca__n_components': 4}
 None

Best Estimator Accuracy: 0.818181818182

Recall Score: 0.2

Precision Score: 0.2
```
Ok, that is not exactly the result we got before but it still shows the data with additional data to perform less well. 

Also, have to run Vivek's suggestion of limiting the depth of the trees to 2 or 3 or setting a minimum sample split because there is so little data. So here is one more time with the max depth set lower. That should not increase the preformance of the model (since our scoring features is selecting the model that performs the best) but it might provide some insight. Also, again following Vivek's suggestion, I will limit the principle component analysis to 10. 
```
Best parameters from the grid search:{'clf__max_depth': 3, 'imp__strategy': 'most_frequent', 'pca__n_components': 3}
 None

Best Estimator Accuracy: 0.75

Recall Score: 0.2

Precision Score: 0.125
```
Yeah, not such hot performance but at least is makes more sense. 

Now I am going to see if the model can work better with the two variables that have no association with the dependent variable, the 'drop' data set. 
```

Best parameters from the grid search:{'clf__max_depth': 3, 'imp__strategy': 'most_frequent', 'pca__n_components': 6}
 None

Best Estimator Accuracy: 0.977272727273

Recall Score: 1.0

Precision Score: 0.833333333333
```
Now, with that as a base line lets see if the Robust Scaler works better. 
```

Best parameters from the grid search:{'clf__max_depth': 3, 'imp__strategy': 'most_frequent', 'pca__n_components': 7}
 None

Best Estimator Accuracy: 0.909090909091

Recall Score: 0.2

Precision Score: 1.0
```
Ok, this is a lot worse. In fact, it is so much worse that I can't believe these results are correct. I must have changed something other than the just the scaler. 

These are the results with the two 0 poi variables dropped and the grid search run on the whole data set. 
```
Best parameters from the grid search:{'clf__max_depth': 2, 'imp__strategy': 'most_frequent', 'pca__n_components': 8}
 None

Best Estimator Accuracy: 0.909090909091

Recall Score: 0.444444444444

Precision Score: 0.888888888889
```




## Results with 1000, 10 and 100 iterations

### 1000 iterations
So the first outcome from the model is pretty good. It is StratifiedShuffleSplit with 1000 iterations. The outcome was really good but it took forever. 

```
Best parameters from the grid search:{'clf__max_depth': 9, 'pca__n_components': 10}
 None

Best Estimator Accuracy: 0.886363636364


Recall Score: 0.6


Precision Score: 0.5
```
### 10 iterations
This is great but I just can't use this for testing things out. It will take forever. So, I am going to try it with 10 iterations and see what happens. 

```

Best parameters from the grid search:{'clf__max_depth': 9, 'pca__n_components': 12}
 None

Best Estimator Accuracy: 0.863636363636


Recall Score: 0.4


Precision Score: 0.4
```
### 100
Now we get the same depth but 12 instead of 10 principle components. The scores have gone down though. It is worth trying it at a higher number of iterations. I am going to try 100. Here is the outcome: 
```
Best Estimator Accuracy: 0.818181818182


Recall Score: 0.4


Precision Score: 0.285714285714
```
Ok, so 100 iterations is worse than either 1000 or 10 iterations. That is kind of distressing. It would be nice if the behavior of the model was 'monotonic', that is, the more of one thing you do the more of something you are looking for you get. I was thinking that 100 iterations could serve as a good way to explore possibilities and narrow the search space while reserving 1000 iteration runs to make the final cut. Now I am not quite sure what to do. 

And here is another thing. I just ran the model again with 1000 iterations with the single addition of the clf's criterion being 'gini' or 'entropy'. It came back with gini as the better criterion. And the model had performance that was just as good as before on the 1000 iterations. In fact, the three scores of interest--accuracy, precision and recall--were exactly the same. But it found a max depth of 7 instead of 9. 
```
Best parameters from the grid search:{'clf__criterion': 'gini', 'clf__max_depth': 7, 'pca__n_components': 10}
 None

Best Estimator Accuracy: 0.886363636364


Recall Score: 0.6


Precision Score: 0.5
```
The reason that is so puzzling to me is that gini is the default scoring criterion for the model. So it was using gini before when it found the best preformance was at max depth of 9 instead of 7. How does that happen?

### Adding variables

In this iteration I try adding some new variables and the choice between the median and most_frequent imputation for the missing values. I also limited the search space for 'max_depth' and the number of components to 6-8 and 8-12 respectively. That may have been a mistake because I had found that the most effective max depth was 9. I don't know why I did that. 
```

Best parameters from the grid search:{'clf__max_depth': 7, 'imp__strategy': 'most_frequent', 'pca__n_components': 8}
 None

Best Estimator Accuracy: 0.886363636364


Recall Score: 0.2


Precision Score: 0.5
```
The results are strange in that adding more variables decreased the preformance, but I can't be sure that it wasn't because of my odd choice of cutting off the max_depth at 8 instead of 9. 

Also, I don't know why the max_depth is a useful parameter since the default is None. How could limiting the parameter improve preformance? 

## New Variables with modified parameter search space

Just to satisfy my curiosity I am going to increase the max_depth parameter to 10 and raise the maximum number of principle components to 16 to account for the four newly added variables.

#### Civil Libertarian Improvement

So this turns out to have been a good idea. I have had a slight loss in recall but big gains in accuracy and precision. Given that I am a civil libertarian I personally feel this is an improvement. 

```

Best parameters from the grid search:{'clf__max_depth': 8, 'imp__strategy': 'most_frequent', 'pca__n_components': 8}
 None

Best Estimator Accuracy: 0.909090909091


Recall Score: 0.4


Precision Score: 0.666666666667
```
Now I have one problem with these results in that the number of principle components chosen was at the bottom of the range I had specified. So now I have to face the possibility that there was better preforming model with a smaller dimensional principle component space that was over looked by the paramters to which I limited the grid search. So, as a double check, I am going to let the space searced go down. I am also going to keep the 'most_frequent' strategy and not search that space anymore. 

### Fewer dimensions: Insanity

Ok, this is the kind of insane behavior that is driving me insane. 
```

Best parameters from the grid search:{'clf__max_depth': 8, 'pca__n_components': 7}
 None

Best Estimator Accuracy: 0.863636363636


Recall Score: 0.2


Precision Score: 0.333333333333
```
Now, all I did was specify 'most_frequent' as the method of imputation and offered it the choice of finding few dimensions in the principle component analysis. Everything else was the same. So how could the model get worse? It had the choice of keeping the model that had preformed better in the last specificaiton of the model, so how could it get worse? 
## Showdown
Ok, I am just going to test everything. I am going to have a model of 200 different possible parameter configurations, but I am going to know once and for all what's what. Since I have never had entropy come up as the better scoring criterion I am going to leave that out but I am going to try everything else I have tried with the broades possible ranges. 

I am doing this essentially because I have to know once and for all whether more variables improve things or not. 
```

Best parameters from the grid search:{'clf__max_depth': 7, 'imp__strategy': 'most_frequent', 'pca__n_components': 8}
 None

Best Estimator Accuracy: 0.886363636364


Recall Score: 0.2


Precision Score: 0.5
```
So by expanding the grid search the model actually does worse. That is really distressing. 

### Sanity Test
So now I have been running into cases where I expand the search grid but end up with a less effective model. This is puzzling. It seems that if I have the models parameters set wider but still including the original model it should do at least as well as the original model. So, I am going to test the proposition and try to recreate the problem. 

First I will run the model that preformed the best above. 

Then I will run a model with a slightly expanded grid search. 

Then I will run a model with a much expanded grid search. 

### Larger Data Set-Original Settings
So I tried it with the original grid search settings, with the choice between 'median' and most_frequent', principle components pared down to the range from 2 to 10 and the max depth set to the range from 6 to 9. The results were pretty good. 
```

Best parameters from the grid search:{'clf__max_depth': 8, 'imp__strategy': 'most_frequent', 'pca__n_components': 8}
 None

Best Estimator Accuracy: 0.909090909091


Recall Score: 0.4


Precision Score: 0.666666666667
```
That is reassuring because it is the exact outcome from the model that I labeled the 'civil libertarian improvement' above. 

What was odd was when I gave it the exact settings that had preformed best in the broadest grid search with the smaller data set: depth 9/components 10, I got the error about something being 0. Then I changed the grid search to have lists for the components and depth parameters of 8,9 and 9,10 respectively so that model would run without objecting that it needed lists in the grid search parameters. The outcome is acceptable but the preformance has gone down since adding new variables. That is really weird. 
```

Best parameters from the grid search:{'clf__max_depth': 8, 'imp__strategy': 'most_frequent', 'pca__n_components': 9}
 None

Best Estimator Accuracy: 0.886363636364


Recall Score: 0.4


Precision Score: 0.5
```
So maybe adding new variables that are simply restatements of the information in the old variables does not really help. Maybe it makes things worse by increasing dimensionality without adding genuinely new information, or at least not new enough information to justify the increase in the dimensionality of the search space. (?)